# Text task notebook template
## Loading the necessary libraries

In [1]:
import os


os.environ['TF_USE_LEGACY_KERAS'] = "1"

In [2]:
from fastapi import APIRouter
from datetime import datetime
from datasets import load_dataset
from sklearn.metrics import accuracy_score
import random

import sys
sys.path.append('../tasks')

from utils.evaluation import AudioEvaluationRequest
from utils.emissions import tracker, clean_emissions_data, get_space_info


# Define the label mapping
LABEL_MAPPING = {
    "chainsaw": 0,
    "environment": 1
}

/home/user/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[codecarbon WARNING @ 08:42:55] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 08:42:55] [setup] RAM Tracking...
[codecarbon INFO @ 08:42:55] [setup] CPU Tracking...
[codecarbon WARNING @ 08:42:55] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 08:42:57] CPU Model on constant consumption mode: Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz
[codecarbon INFO @ 08:42:57] [setup] GPU Tracking...
[codecarbon INFO @ 08:42:57] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 08:42:57] >>> Tracker's metadata:
[codecarbon INFO @ 08:42:57]   Platform system: Linux-5.10.23

In [3]:
import cmsisdsp

In [4]:
import cmsisdsp
import numpy as np
import tensorflow as tf
from numpy import pi as PI

window_size = 256
step_size = 128*8

hanning_window_f32 = np.zeros(window_size)
for i in range(window_size):
  hanning_window_f32[i] = 0.5 * (1 - cmsisdsp.arm_cos_f32(2 * PI * i / window_size ))

hanning_window_q15 = cmsisdsp.arm_float_to_q15(hanning_window_f32)

rfftq15 = cmsisdsp.arm_rfft_instance_q15()
status = cmsisdsp.arm_rfft_init_q15(rfftq15, window_size, 0, 1)

def get_arm_spectrogram(waveform):

  num_frames = int(1 + (len(waveform) - window_size) // step_size)
  fft_size = int(window_size // 2 + 1)
  # normalisation du son
  waveform = 8*256.0 * (waveform/np.max(np.abs(waveform))-np.mean(waveform))
  # Convert the audio to q15
  waveform_q15 = cmsisdsp.arm_float_to_q15(waveform)
  #print(waveform_q15)
  # Create empty spectrogram array
  spectrogram_q15 = np.empty((num_frames, fft_size), dtype = np.int16)

  start_index = 0

  for index in range(num_frames):
    # Take the window from the waveform.
    window = waveform_q15[start_index:start_index + window_size]

    # Apply the Hanning Window.
    window = cmsisdsp.arm_mult_q15(window, hanning_window_q15)

    # Calculate the FFT, shift by 7 according to docs
    window = cmsisdsp.arm_rfft_q15(rfftq15, window)

    # Take the absolute value of the FFT and add to the Spectrogram.
    spectrogram_q15[index] = cmsisdsp.arm_cmplx_mag_q15(window)[:fft_size]

    # Increase the start index of the window by the overlap amount.
    start_index += step_size

  # Convert to numpy output ready for keras
  OUT = cmsisdsp.arm_q15_to_float(spectrogram_q15).reshape(num_frames,fft_size) * 512
  AVG = np.average(OUT[0,110:])
  MAX = np.max(OUT)

  OUT = OUT/ MAX
  return tf.reshape(OUT, shape=(35, 129))

2025-01-30 08:43:00.778804: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-30 08:43:00.781470: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-30 08:43:00.785511: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-30 08:43:00.795078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738226580.814136    6130 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738226580.81

In [5]:
import tensorflow.lite as tflite

# Load the interpreter and allocate tensors
interpreter = tflite.Interpreter( f"../models/bm_v11.tflite")
interpreter.allocate_tensors()

# Load input and output details
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

# Set quantization values
input_scale, input_zero_point = input_details["quantization"]
output_scale, output_zero_point = output_details["quantization"]

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Loading the datasets and splitting them

In [6]:
request = AudioEvaluationRequest()

print(request.test_size, request.test_seed)
# Load and prepare the dataset
dataset = load_dataset(request.dataset_name,cache_dir="cache/")

# Split dataset
train_test = dataset["train"].train_test_split(test_size=request.test_size, seed=request.test_seed)
test_dataset = train_test["test"]

0.2 42


In [7]:
len(test_dataset)

for example in test_dataset.select(range(10)):
    WAV = example["audio"]["array"]
    
    get_arm_spectrogram(WAV)

W0000 00:00:1738226585.018929    6130 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Random Baseline

In [8]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

[]


In [9]:


# Define a generator function for the dataset
def dataset_generator(dataset):
    for example in dataset:
        audio_array = example['audio']['array']
        label = example['label']
        # Randomly sample 1-5 integers
        fold = random.choice([0, 1, 2, 3, 4])
        yield audio_array, label, fold

def create_arm_spectrogram_for_map(samples, label, fold):
    spectrogram = tf.py_function(get_arm_spectrogram, [samples], tf.float32)
    return spectrogram, label, fold

def remove_fold_column(spectrogram, label, fold):
    spectrogram = tf.expand_dims(spectrogram, axis=-1)
    spectrogram.set_shape([35, 129, 1])
    print("The spectrogram Shape", spectrogram.shape)
    return spectrogram, label

def filter_by_length(audio, label, path):
    return tf.equal(tf.shape(audio)[0], 36000)
    
def create_testds(DS):
    # Convert the dataset to a TensorFlow dataset
  test_ds = tf.data.Dataset.from_generator(
      lambda: dataset_generator(DS),
      output_signature=(
          tf.TensorSpec(shape=(None,), dtype=tf.float32),  # Audio array
          tf.TensorSpec(shape=(), dtype=tf.int64),         # Label
          tf.TensorSpec(shape=(), dtype=tf.int64)         # Path
      )
  )

  
  test_ds = test_ds.filter(filter_by_length)
  test_ds = test_ds.map(create_arm_spectrogram_for_map)
  test_ds = test_ds.map(remove_fold_column)
  test_ds = test_ds.cache().batch(32, drop_remainder=True).prefetch(tf.data.AUTOTUNE).cache()

  return test_ds



def evalz(test_ds):
  results = []
  correct = 0
  test_ds_len = 0
  for x, y in  test_ds.cache().unbatch():
      # original tdrishape is [124, 129, 1] expand to [1, 124, 129, 1]
      x = tf.expand_dims(x, 0).numpy()
      #if not test_ds_len%10:
      #  print(test_ds_len)
      # quantize the input value
      if (input_scale, input_zero_point) != (0, 0):
        x = x / input_scale + input_zero_point

      X = np.float32(x)
      X = X.astype(input_details['dtype'])

      try:
        # add the input tensor to interpreter
        interpreter.set_tensor(input_details["index"], X)

        #run the model
        interpreter.invoke()

        # Get output data from model and convert to fp32
        output_data = interpreter.get_tensor(output_details["index"])
        output_data = output_data.astype(np.float32)

        # Dequantize the output
        if (output_scale, output_zero_point) != (0.0, 0):
          output_data = (output_data - output_zero_point) * output_scale

        # convert output to category
        if output_data[0][0] >= 0.5:
          category = 0
        else:
          category = 1
      except:
         category  = random.randint(0, 1)

      # add 1 if category = y
      correct += 1 if category == y.numpy() else 0
      test_ds_len += 1

      results.append(category)

  return results, correct, test_ds_len

In [11]:
test_dataset = train_test["test"]
test_ds = create_testds(test_dataset)
#test_ds = test_ds.map(create_arm_spectrogram_for_map)
test_ds.element_spec

The spectrogram Shape (35, 129, 1)


(TensorSpec(shape=(32, 35, 129, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(32,), dtype=tf.int64, name=None))

In [12]:
test_ds.take(1).unbatch()


<_UnbatchDataset element_spec=(TensorSpec(shape=(35, 129, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [13]:
# Start tracking emissions
tracker.start()
tracker.start_task("inference")

In [14]:

#--------------------------------------------------------------------------------------------
# YOUR MODEL INFERENCE CODE HERE
# Update the code below to replace the random baseline by your model inference within the inference pass where the energy consumption and emissions are tracked.
#--------------------------------------------------------------------------------------------   

# Make random predictions (placeholder for actual model inference)

predictions, correct, test_ds_len = evalz(test_ds)

#--------------------------------------------------------------------------------------------
# YOUR MODEL INFERENCE STOPS HERE
#--------------------------------------------------------------------------------------------   

2025-01-30 08:44:10.778935: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [15]:
# Stop tracking emissions
emissions_data = tracker.stop_task()
emissions_data

[codecarbon INFO @ 08:44:10] Energy consumed for RAM : 0.000038 kWh. RAM Power : 5.74672794342041 W
[codecarbon INFO @ 08:44:10] Energy consumed for all CPUs : 0.000692 kWh. Total CPU Power : 105.0 W
[codecarbon INFO @ 08:44:10] Energy consumed for all GPUs : 0.000092 kWh. Total GPU Power : 13.931956061721136 W
[codecarbon INFO @ 08:44:10] 0.000821 kWh of electricity used since the beginning.


EmissionsData(timestamp='2025-01-30T08:44:10', project_name='codecarbon', run_id='91894b0b-b059-41ec-b674-633c1537a041', experiment_id='5b0fa12a-3dd7-45bb-9766-cc326314d9f1', duration=23.716542253000625, emissions=0.0003032031842741412, emissions_rate=1.278461399935696e-05, cpu_power=105.0, gpu_power=13.931956061721136, ram_power=5.74672794342041, cpu_energy=0.0006917466789333464, gpu_energy=9.17864623179998e-05, ram_energy=3.785658139292668e-05, energy_consumed=0.0008213897226442729, country_name='United States', country_iso_code='USA', region='virginia', cloud_provider='', cloud_region='', os='Linux-5.10.230-223.885.amzn2.x86_64-x86_64-with-glibc2.36', python_version='3.11.11', codecarbon_version='2.8.3', cpu_count=4, cpu_model='Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz', gpu_count=1, gpu_model='1 x Tesla T4', longitude=-77.4903, latitude=39.0469, ram_total_size=15.324607849121094, tracking_mode='machine', on_cloud='N', pue=1.0)

In [16]:
accuracy = correct / test_ds_len
accuracy

0.8713235294117647

In [17]:
# Prepare results dictionary
results = {
    "submission_timestamp": datetime.now().isoformat(),
    "accuracy": float(accuracy),
    "energy_consumed_wh": emissions_data.energy_consumed * 1000,
    "emissions_gco2eq": emissions_data.emissions * 1000,
    "emissions_data": clean_emissions_data(emissions_data),
    "dataset_config": {
        "dataset_name": request.dataset_name,
        "test_size": request.test_size,
        "test_seed": request.test_seed
    }
}

results

{'submission_timestamp': '2025-01-30T08:44:10.806702',
 'accuracy': 0.8713235294117647,
 'energy_consumed_wh': 0.8213897226442729,
 'emissions_gco2eq': 0.30320318427414117,
 'emissions_data': {'run_id': '91894b0b-b059-41ec-b674-633c1537a041',
  'duration': 23.716542253000625,
  'emissions': 0.0003032031842741412,
  'emissions_rate': 1.278461399935696e-05,
  'cpu_power': 105.0,
  'gpu_power': 13.931956061721136,
  'ram_power': 5.74672794342041,
  'cpu_energy': 0.0006917466789333464,
  'gpu_energy': 9.17864623179998e-05,
  'ram_energy': 3.785658139292668e-05,
  'energy_consumed': 0.0008213897226442729,
  'country_name': 'United States',
  'country_iso_code': 'USA',
  'region': 'virginia',
  'cloud_provider': '',
  'cloud_region': '',
  'os': 'Linux-5.10.230-223.885.amzn2.x86_64-x86_64-with-glibc2.36',
  'python_version': '3.11.11',
  'codecarbon_version': '2.8.3',
  'cpu_count': 4,
  'cpu_model': 'Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz',
  'gpu_count': 1,
  'gpu_model': '1 x Tesl